In [1]:
import pandas as pd
import numpy as np

from pycta.portfolio import Portfolio
from pycta.beakerx import *
from pycta.signal import *

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Load prices
prices = pd.read_csv("data/prices.csv", index_col=0, parse_dates=True).ffill().truncate(before="1970-01-01")

In [3]:
mu = np.tanh(prices.apply(returns_adjust, com=32, clip=4.2).cumsum().apply(osc))
vola = prices.pct_change().ewm(com=32, min_periods=32).std()

# compute the series of Euclidean norms by compute the sum of squares for each row
euclid_norm = np.sqrt((mu*mu).sum(axis=1))

# Divide each column of mu by the Euclidean norm
risk_scaled = mu.apply(lambda x: x/euclid_norm, axis=0)

portfolio = Portfolio(prices=prices, position=risk_scaled/vola)

In [4]:
analysis(portfolio.nav())

# Conclusions
* Diagonal Markowitz already helps to improve the Sharpe Ratio. 
* The risk position is always scaled such that the risk is constant. This helps in particular to bring down the Kurtosis.
* The risk however does not reflect any cross-correlations.